## smile detection and taking pictures

In [17]:
import cv2
import time
import os
import pyttsx3

# Load Haar cascade classifiers for face, eyes, and smile detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')
smile_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_smile.xml')

# Initialize text-to-speech engine
def speak(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

stable_eye_count = {}
stable_smile_count = {}
STABLE_THRESHOLD = 10  # Require detections over multiple frames

def detect_faces(frame):
    global stable_eye_count, stable_smile_count
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(100, 100))
    
    detected_faces = 0
    smiling_faces = 0
    
    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = frame[y:y+h, x:x+w]
        
        # Detect eyes and smiles
        eyes = eye_cascade.detectMultiScale(roi_gray, scaleFactor=1.1, minNeighbors=6)
        smiles = smile_cascade.detectMultiScale(roi_gray, scaleFactor=1.5, minNeighbors=25)
        
        face_id = f"face_{detected_faces}"
        detected_faces += 1
        
        # Initialize counts if not present
        if face_id not in stable_eye_count:
            stable_eye_count[face_id] = 0
        if face_id not in stable_smile_count:
            stable_smile_count[face_id] = 0
        
        # Update counts based on detection stability
        stable_eye_count[face_id] = max(0, stable_eye_count[face_id] - 1) if len(eyes) < 2 else stable_eye_count[face_id] + 1
        stable_smile_count[face_id] = max(0, stable_smile_count[face_id] - 1) if len(smiles) == 0 else stable_smile_count[face_id] + 1
        
        print(f"Person {detected_faces} - Eyes: {stable_eye_count[face_id]}, Smile: {stable_smile_count[face_id]}")
        
        # Determine colors for rectangles
        face_color = (0, 255, 0) if stable_eye_count[face_id] >= STABLE_THRESHOLD and stable_smile_count[face_id] >= STABLE_THRESHOLD else (0, 0, 255)
        smile_color = (0, 255, 0) if stable_smile_count[face_id] >= STABLE_THRESHOLD else (0, 0, 255)
        
        # Draw face rectangle
        cv2.rectangle(frame, (x, y), (x+w, y+h), face_color, 2)  
        
        # Draw smile rectangle if detected
        for (sx, sy, sw, sh) in smiles:
            cv2.rectangle(roi_color, (sx, sy), (sx+sw, sy+sh), smile_color, 2)  # Rectangle over the mouth
        
        if stable_eye_count[face_id] >= STABLE_THRESHOLD and stable_smile_count[face_id] >= STABLE_THRESHOLD:
            smiling_faces += 1
    
    return frame, detected_faces, smiling_faces

def main():
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)
    cap.set(cv2.CAP_PROP_FPS, 30)  # Increase FPS for smoother capture
    
    pic_count = 1
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Make a copy of the original frame before drawing rectangles
        clean_frame = frame.copy()
        
        frame, detected_faces, smiling_faces = detect_faces(frame)
        
        cv2.imshow('Face Detection', frame)
        
        # Check if all detected faces are smiling
        if detected_faces > 0 and detected_faces == smiling_faces:  
            speak("3, 2, 1, Cheese!")
            
            # Save the clean frame (without rectangles)
            filename = f"captured_image_{pic_count}.jpg"
            cv2.imwrite(filename, clean_frame)
            
            print(f"Console Output: Picture captured and saved as {filename}!")
            speak("Picture taken!")
            break  # Exit after taking the picture
        
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q') or key == 27:  # Press 'q' or ESC to exit
            print("Console Output: Exiting...")
            break
    
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


Person 1 - Eyes: 1, Smile: 0
Person 1 - Eyes: 0, Smile: 0
Person 1 - Eyes: 0, Smile: 0
Person 1 - Eyes: 0, Smile: 0
Person 1 - Eyes: 0, Smile: 0
Person 1 - Eyes: 0, Smile: 0
Person 1 - Eyes: 0, Smile: 1
Person 1 - Eyes: 0, Smile: 0
Person 1 - Eyes: 0, Smile: 0
Person 1 - Eyes: 0, Smile: 0
Person 1 - Eyes: 0, Smile: 0
Person 1 - Eyes: 0, Smile: 0
Person 2 - Eyes: 0, Smile: 0
Person 1 - Eyes: 0, Smile: 0
Person 1 - Eyes: 0, Smile: 0
Person 1 - Eyes: 0, Smile: 1
Person 1 - Eyes: 0, Smile: 2
Person 1 - Eyes: 0, Smile: 1
Person 2 - Eyes: 0, Smile: 0
Person 1 - Eyes: 0, Smile: 0
Person 1 - Eyes: 0, Smile: 0
Person 1 - Eyes: 0, Smile: 0
Person 1 - Eyes: 0, Smile: 0
Person 1 - Eyes: 0, Smile: 0
Person 1 - Eyes: 0, Smile: 0
Person 2 - Eyes: 0, Smile: 0
Person 1 - Eyes: 0, Smile: 1
Person 1 - Eyes: 0, Smile: 0
Person 2 - Eyes: 0, Smile: 0
Person 1 - Eyes: 0, Smile: 0
Person 2 - Eyes: 0, Smile: 0
Person 1 - Eyes: 0, Smile: 0
Person 1 - Eyes: 0, Smile: 0
Person 1 - Eyes: 0, Smile: 0
Person 2 - Eye

In [18]:
import cv2  

# Load pre-trained face detection model  
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')  

# Open webcam  
cap = cv2.VideoCapture(0)  

while True:  
    ret, frame = cap.read()  
    if not ret:  
        break  

    # Convert to grayscale for better accuracy  
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  

    # Detect faces  
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(100, 100))  

    for (x, y, w, h) in faces:  
        # Draw bounding box around the face  
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)  

        # Add label  
        cv2.putText(frame, "Face Detected", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)  

    # Show output  
    cv2.imshow("Face Detection", frame)  

    # Press 'q' to exit  
    if cv2.waitKey(1) & 0xFF == ord('q'):  
        break  

cap.release()  
cv2.destroyAllWindows()  


In [19]:
from ultralytics import YOLO
import cv2

# Load YOLO model (Ensure you have the correct YOLOv8 model)
model = YOLO("yolov8n.pt")  

# Load image
image_path = "captured_image_1.jpg"
image = cv2.imread(image_path)

# Run YOLO detection
results = model(image)

# Process results and display output
for result in results:
    # Get annotated image (YOLO's `plot()` method adds bounding boxes)
    annotated_frame = result.plot()

    # Display the image with detections
    cv2.imshow("YOLOv8 Detection", annotated_frame)
    cv2.waitKey(0)  # Wait for a key press
    cv2.destroyAllWindows()  # Close the display window



0: 384x640 1 person, 181.4ms
Speed: 6.7ms preprocess, 181.4ms inference, 2.3ms postprocess per image at shape (1, 3, 384, 640)


### Saving images of the person

In [20]:
import cv2
import time
import pyttsx3
import os

# Load Haar cascade classifiers for face and eyes detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

# Text-to-speech engine
def speak(text):
    engine = pyttsx3.init()
    engine.say(text)
    engine.runAndWait()

stable_eye_count = {}
STABLE_THRESHOLD = 10  # Require detections over multiple frames

# Get the person's name
person_name = input("Enter your name: ")
base_folder = f"dataset/{person_name}"
os.makedirs(base_folder, exist_ok=True)  # Create a main folder for the person

def detect_faces(frame):
    global stable_eye_count
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(100, 100))
    
    detected_faces = 0
    face_regions = []
    
    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]
        eyes = eye_cascade.detectMultiScale(roi_gray, scaleFactor=1.1, minNeighbors=6)
        
        face_id = f"face_{detected_faces}"
        detected_faces += 1
        
        # Initialize counts if not present
        if face_id not in stable_eye_count:
            stable_eye_count[face_id] = 0
        
        # Update counts based on detection stability
        stable_eye_count[face_id] = max(0, stable_eye_count[face_id] - 1) if len(eyes) < 2 else stable_eye_count[face_id] + 1
        
        print(f"Person {detected_faces} - Eyes: {stable_eye_count[face_id]}")
        
        # Determine rectangle color
        face_color = (0, 255, 0) if stable_eye_count[face_id] >= STABLE_THRESHOLD else (0, 0, 255)
        
        # Draw face rectangle
        cv2.rectangle(frame, (x, y), (x+w, y+h), face_color, 2)
        
        # Display person's name above the rectangle if green
        if face_color == (0, 255, 0):
            cv2.putText(frame, person_name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
            face_regions.append((x, y, w, h))
    
    return frame, detected_faces, face_regions

def main():
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1920)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 1080)
    cap.set(cv2.CAP_PROP_FPS, 30)  # Increase FPS for smoother capture
    
    pic_count = 0
    max_pics = 10
    
    while pic_count < max_pics:
        ret, frame = cap.read()
        if not ret:
            break
        
        frame, detected_faces, face_regions = detect_faces(frame)
        
        cv2.imshow('Face Detection', frame)
        
        # Capture only if detected faces are within the green rectangle
        if detected_faces > 0:
            for (x, y, w, h) in face_regions:
                face_crop = frame[y:y+h, x:x+w]  # Crop only the detected face
                filename = f"{base_folder}/{person_name}_{pic_count}.jpg"
                cv2.imwrite(filename, face_crop)
                print(f"Picture captured: {filename}")
                pic_count += 1
                
                if pic_count >= max_pics:
                    break  # Stop once 10 images are captured
        
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q') or key == 27:  # Press 'q' or ESC to exit
            print("Exiting...")
            break
    
    cap.release()
    cv2.destroyAllWindows()
    speak("All 10 pictures have been taken successfully!")

if __name__ == "__main__":
    main()


Enter your name:  Kavish.


Person 1 - Eyes: 0
Person 1 - Eyes: 0
Person 1 - Eyes: 1
Person 1 - Eyes: 2
Person 1 - Eyes: 3
Person 1 - Eyes: 4
Person 1 - Eyes: 5
Person 1 - Eyes: 6
Person 1 - Eyes: 7
Person 1 - Eyes: 8
Person 1 - Eyes: 7
Person 1 - Eyes: 8
Person 1 - Eyes: 9
Person 1 - Eyes: 10
Picture captured: dataset/Kavish./Kavish._0.jpg
Person 1 - Eyes: 11
Picture captured: dataset/Kavish./Kavish._1.jpg
Person 1 - Eyes: 12
Picture captured: dataset/Kavish./Kavish._2.jpg
Person 1 - Eyes: 13
Picture captured: dataset/Kavish./Kavish._3.jpg
Person 1 - Eyes: 14
Picture captured: dataset/Kavish./Kavish._4.jpg
Person 1 - Eyes: 15
Picture captured: dataset/Kavish./Kavish._5.jpg
Person 1 - Eyes: 16
Picture captured: dataset/Kavish./Kavish._6.jpg
Person 1 - Eyes: 17
Picture captured: dataset/Kavish./Kavish._7.jpg
Person 1 - Eyes: 16
Picture captured: dataset/Kavish./Kavish._8.jpg
Person 1 - Eyes: 17
Picture captured: dataset/Kavish./Kavish._9.jpg


### Face detecting 

In [22]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# ===================== Load & Preprocess Data =====================

# Path to dataset
dataset_path = "dataset"  # Update this path

# Define image size
img_size = 100  # Resize images to 100x100

# Lists to store images and labels
images = []
labels = []

# Load images and labels
for person_name in os.listdir(dataset_path):
    person_path = os.path.join(dataset_path, person_name)
    
    if os.path.isdir(person_path):  # Ensure it's a folder
        for img_name in os.listdir(person_path):
            img_path = os.path.join(person_path, img_name)
            img = cv2.imread(img_path)
            
            if img is not None:
                img = cv2.resize(img, (img_size, img_size))  # Resize image
                images.append(img)
                labels.append(person_name)  # Use folder name as label

# Convert lists to numpy arrays
images = np.array(images) / 255.0  # Normalize pixel values
labels = np.array(labels)

# Encode labels
label_encoder = LabelEncoder()
labels_encoded = label_encoder.fit_transform(labels)
num_classes = len(np.unique(labels_encoded))

# Convert labels to categorical (for softmax output)
labels_categorical = to_categorical(labels_encoded, num_classes)

# Ensure consistency in shape
assert images.shape[0] == labels_categorical.shape[0], "Mismatch between images and labels"

# ===================== Train-Test Split =====================

X_train, X_test, y_train, y_test = train_test_split(images, labels_categorical, test_size=0.2, random_state=42)

# ===================== Build CNN Model =====================

model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_size, img_size, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')  # Output layer
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# ===================== Train Model =====================

model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

# Save trained model
model.save("face_recognition_model.h5")
np.save("label_classes.npy", label_encoder.classes_)  # Save label classes

print("Model training completed and saved!")

# ===================== Real-Time Face Detection =====================

# Load trained model
model = tf.keras.models.load_model("face_recognition_model.h5")
label_encoder.classes_ = np.load("label_classes.npy", allow_pickle=True)

# Load OpenCV face detector
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Start webcam
cap = cv2.VideoCapture(0)

# Set full-screen mode
cv2.namedWindow("Face Recognition", cv2.WND_PROP_FULLSCREEN)
cv2.setWindowProperty("Face Recognition", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        face = frame[y:y+h, x:x+w]  # Extract face
        face = cv2.resize(face, (img_size, img_size)) / 255.0  # Preprocess
        face = np.expand_dims(face, axis=0)  # Reshape for CNN
        
        prediction = model.predict(face)
        class_index = np.argmax(prediction)
        person_name = label_encoder.inverse_transform([class_index])[0]  # Decode label
        
        # Draw rectangle & label
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
        cv2.putText(frame, person_name, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Face Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Epoch 1/10
4/4 [==============================] - 5s 591ms/step - loss: 1.7554 - accuracy: 0.3853 - val_loss: 1.5224 - val_accuracy: 0.5000
Epoch 2/10
4/4 [==============================] - 2s 542ms/step - loss: 1.5604 - accuracy: 0.5413 - val_loss: 1.4880 - val_accuracy: 0.5000
Epoch 3/10
4/4 [==============================] - 2s 579ms/step - loss: 1.5056 - accuracy: 0.5413 - val_loss: 1.4484 - val_accuracy: 0.5000
Epoch 4/10
4/4 [==============================] - 2s 451ms/step - loss: 1.5272 - accuracy: 0.5413 - val_loss: 1.3896 - val_accuracy: 0.5000
Epoch 5/10
4/4 [==============================] - 2s 481ms/step - loss: 1.4385 - accuracy: 0.5413 - val_loss: 1.3871 - val_accuracy: 0.5000
Epoch 6/10
4/4 [==============================] - 2s 556ms/step - loss: 1.3807 - accuracy: 0.5413 - val_loss: 1.3599 - val_accuracy: 0.5000
Epoch 7/10
4/4 [==============================] - 2s 482ms/step - loss: 1.3243 - accuracy: 0.5413 - val_loss: 1.2174 - val_accuracy: 0.5000
Epoch 8/10
4/4 [====

C:\Users\22ag5\anaconda3\envs\financial\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1/1 [==============================] - 0s 34ms/step
